In [1]:
import pandas as pd
import numpy as np

In [5]:
# Cargar los datos de ventas desde un archivo Excel sin encabezados
# Cargar sin encabezado es crucial porque las columnas necesitan ser reparadas
datos_ventas = pd.read_excel('https://foresightbi.com.ng/wp-content/uploads/2020/05/3.-Badly-Structured-Sales-Data-3.xlsx', header=None)

# Eliminar la última fila, que contiene un resumen o datos no deseados
datos_ventas = datos_ventas.iloc[:-1]

datos_ventas.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,NaN,Ship Mode,First Class,NaN,NaN,Same Day,NaN,NaN,Second Class,NaN,NaN,Standard Class,NaN,NaN
1,NaN,Segment,Consumer,Corporate,Home Office,Consumer,Corporate,Home Office,Consumer,Corporate,Home Office,Consumer,Corporate,Home Office
2,Order ID,Order Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CA-2011-100293,2013-03-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.056
4,CA-2011-100706,2013-12-16 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,129.44,NaN,NaN,NaN,NaN,NaN


In [7]:
# Definir los nombres de las columnas basados en los modos de envío y sus respectivos conteos
# Si miras las dos primeras filas en el archivo Excel, verás que para cada modo de envío
# hay tres segmentos: Consumidor, Corporativo y Oficina en Casa, por lo que he replicado
# cada modo de envío tres veces
modos_envio = (
    ['Primera Clase'] * 3 + 
    ['Mismo Día'] * 3 + 
    ['Segunda Clase'] * 3 + 
    ['Clase Estándar'] * 3
)

modos_envio[:10]


['Primera Clase',
 'Primera Clase',
 'Primera Clase',
 'Mismo Día',
 'Mismo Día',
 'Mismo Día',
 'Segunda Clase',
 'Segunda Clase',
 'Segunda Clase',
 'Clase Estándar']

In [8]:
# Establecer la primera fila con los nombres de columnas apropiados
datos_ventas.iloc[0] = [np.nan, 'Modo de Envío'] + modos_envio

# Transponer el DataFrame y omitir las primeras dos filas
datos_transpuestos = datos_ventas.T.iloc[2:]

# Crear un diccionario para almacenar los datos estructurados de ventas
datos_ventas_estructurados = dict()

# Llenar el diccionario con los detalles de los pedidos
for indice, fila in datos_transpuestos.iterrows():
    valores_no_na = fila[fila.notna()]
    datos_ventas_estructurados[(valores_no_na[0], valores_no_na[1])] = [
        (indice_columna, valor) for indice_columna, valor in zip(valores_no_na.index[2:], valores_no_na.values[2:])
    ]

# Inicializar listas para almacenar los detalles de los pedidos extraídos
ids_pedidos = []
fechas_pedidos = []
modos_envio = []
segmentos_clientes = []
precios = []

# Extraer los detalles de los pedidos desde los datos estructurados de ventas
for (modo_envio, segmento), valores in datos_ventas_estructurados.items():
    for indice_columna, precio in valores:
        id_pedido, fecha_pedido = datos_ventas.iloc[indice_columna, :2]
        fechas_pedidos.append(fecha_pedido)
        ids_pedidos.append(id_pedido)
        modos_envio.append(modo_envio)
        segmentos_clientes.append(segmento)
        precios.append(precio)

# Crear un nuevo DataFrame con los detalles estructurados de los pedidos
datos_pedidos_estructurados = pd.DataFrame({
    'ID del Pedido': ids_pedidos,
    'Fecha del Pedido': fechas_pedidos,
    'Segmento': segmentos_clientes,
    'Modo de Envío': modos_envio,
    'Precio': precios
})

# Mostrar las primeras filas de los datos estructurados de los pedidos
datos_pedidos_estructurados.head()


,ID del Pedido,Fecha del Pedido,Segmento,Modo de Envío,Precio
0,CA-2011-103366,2013-01-15,Consumer,Primera Clase,149.950
1,CA-2011-109043,2013-08-15,Consumer,Primera Clase,243.600
2,CA-2011-113166,2013-12-24,Consumer,Primera Clase,9.568
3,CA-2011-124023,2013-04-07,Consumer,Primera Clase,8.960
4,CA-2011-130155,2013-05-19,Consumer,Primera Clase,34.200
